In [ ]:
import pandas as pd
import sqlite3

In [ ]:
db_path = 'db.sqlite'
conn = sqlite3.connect(db_path)
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print(tables)

In [ ]:
df_biography = pd.read_sql_query("SELECT * FROM biography", conn)
df_friend = pd.read_sql_query("SELECT * FROM friend", conn)
df_group_table = pd.read_sql_query("SELECT * FROM group_table", conn)
df_photo = pd.read_sql_query("SELECT * FROM photo", conn)
df_post = pd.read_sql_query("SELECT * FROM post", conn)
df_user = pd.read_sql_query("SELECT * FROM user", conn)

In [ ]:
import pandas as pd
import numpy as np

# Создаем DataFrame с нужными столбцами
df_numbers = pd.DataFrame(columns=["id", 'user_id', 'age', 'cnt_post', 'cnt_photo', 'cnt_group', 'cnt_friend', 'post_dates'])

# Для каждого пользователя из df_user
for i, row in df_user.iterrows():
    user_id = row['vk_id']
    id = row['id']

    # Создаем пустую запись для пользователя
    new_row = {
        "id": id,
        'user_id': user_id,
        'cnt_post': 0,
        #'average_post_per_month': 0,
        'cnt_photo': 0,
        'cnt_group': 0,
        'cnt_friend': 0,
        'post_dates': []
    }
    
    age = df_user[df_user['vk_id'] == user_id]['age']
    try:
        if (int(age.iloc[0]) >= 14 and int(age.iloc[0]) <= 17):
            new_row['age'] = 1
        elif (int(age.iloc[0]) >= 18 and int(age.iloc[0]) <= 21):
            new_row['age'] = 2
        elif (int(age.iloc[0]) >= 22 and int(age.iloc[0]) <= 25):
            new_row['age'] = 3
    except:
        continue

    # Подсчет друзей для user_id
    new_row['cnt_friend'] = len(df_friend[df_friend['user_id'] == id])

    # Подсчет постов и сбор дат постов
    user_posts = df_post[df_post['user_id'] == id]
    new_row['cnt_post'] = len(user_posts)
    new_row['post_dates'] = list(user_posts['date'])

    # Подсчет фотографий для user_id
    new_row['cnt_photo'] = len(df_photo[df_photo['user_id'] == id])

    # Подсчет групп для user_id
    new_row['cnt_group'] = len(df_group_table[df_group_table['user_id'] == id])

    # Добавляем новую строку в df_numbers
    df_numbers.loc[len(df_numbers)] = new_row

# Подсчет среднего количества постов в месяц (пример для 12 месяцев)
#df_numbers['average_post_per_month'] = df_numbers['cnt_post'] / 12

df_numbers.info()

In [ ]:
import requests
import time
access_token = 'YOUR_TOKEN'
user_list = list(df_numbers['user_id'])

def getjson(url, data=None):
    response = requests.get(url, params=data)
    response = response.json()
    return response

def get_all_posts(access_token, owner_id, count=100, offset=0):
    all_posts = []
    while True:
        try:
            wall = getjson('https://api.vk.com/method/wall.get',
                            {'owner_id': owner_id, 'offset': offset, 'count': count, 'access_token': access_token,
                            'v': '5.131'})
            posts = wall['response']['items']
        except:
            return 0

        all_posts.extend(posts)

        if len(wall['response']['items']) == 0:
            break
        else:
            offset += 100
    return all_posts

def get_group_name(owner_id):
    url = "https://api.vk.com/method/groups.getById"
    params = {
        "group_id": owner_id,
        "access_token": access_token,
        "v": "5.131"
    }
    response = requests.get(url, params=params)
    group_data = response.json()
    group_name = group_data["response"][0]["name"]

    return group_name

def get_all_liked_lists(access_token, owner_id, liked_object_id, count=1000, offset=0, friends_only=0):
    time.sleep(0.03)
    try:
        api_query = getjson('https://api.vk.com/method/likes.getList',
                            {'access_token': access_token, 'type': 'post', 'owner_id': int(owner_id), 'item_id': liked_object_id,
                                'filter': 'likes', 'friends_only': friends_only, 'count': count, 'v': '5.131'})
        Users_count = api_query['response']['count']
        List_of_users = api_query['response']['items']
    except:
        return 0, 0
    return Users_count, List_of_users

def get_all_sub(access_token, user_id, count=30, offset=0):
    time.sleep(0.03)
    subs = []
    try:
        api_query = getjson('https://api.vk.com/method/users.getSubscriptions',
                            {'access_token': access_token, 'extended': 1, 'user_id': user_id, 'offset': offset, 'count': count, 'v': '5.131'})
        sub = api_query['response']['items']
        subs.extend(sub)
    except:
        return 0  
    return subs

url = 'https://api.vk.com/method/wall.get'
params = {
    "domain": url.split("/")[-1],
    "access_token": access_token,
    "v": "5.131"
}
flag = 0
flag_two = 0
flag_end = 0

# Получение постов, которые пользователь лайкнул с поддержкой пагинации
def get_liked_posts(access_token, owner_id, count=100, offset=0):
    all_posts = []
    while True:
        try:
            wall = getjson('https://api.vk.com/method/fave.get',
                            {'item_type': 'post', 'tag_id': owner_id, 'offset': offset, 'count': count, 'access_token': access_token,
                            'v': '5.131'})
            print(wall)
            posts = wall['response']['items']
        except:
            return 0

        all_posts.extend(posts)

        if len(wall['response']['items']) == 0:
            break
        else:
            offset += 100
    return all_posts

print(user_list)
for i in user_list:
    flag_end += 1
    if (flag_end == 5001):
        break
    subs = get_all_sub(access_token, i)
    if subs == 0:
        continue
    print(f'User: {flag_end}')
    user_liked_posts = get_liked_posts(access_token, i)

    # Собираем массив текстов лайкнутых постов
    liked_texts = [post['text'] for post in user_liked_posts if post['text']]

    df_numbers.loc[df_numbers['user_id'] == i, 'cnt_likes'] += len(user_liked_posts)
    df_numbers.loc[df_numbers['user_id'] == i, 'liked_text'] = liked_texts

    time.sleep(0.1)  # Задержка между запросами, чтобы не превысить лимиты AP

    for j in subs:
        posts = get_all_posts(access_token, j['id'])
        if posts == 0 or posts == []:
            continue
        for k in posts:
            cnt_likes, list_of_users = get_all_liked_lists(access_token, j['id'], k['id'])
            if (cnt_likes == 0):
                continue
            if (int(i) in list_of_users):
                print('YEAH')
                df_numbers.loc[df_numbers['user_id'] == i, 'cnt_likes'] += 1
                df_numbers.loc[df_numbers['user_id'] == i, 'liked_text'] = k['text']
                flag = 0
            else:                  
                flag += 1
                if flag == 7:
                    print('flag inrupt')
                    flag = 0
                    break